In [63]:
import os, json
import pandas as pd
import enum 
import statsmodels.stats.multitest as sm
from src.utils import sort_by_chromosome, RESOLUTIONS
from src .cluster_description import Cluster, ClustersDescription

class ValueKeys(enum.Enum):
    CHROMOSOME = "chromosome"
    NAME = "name"
    P_VALUE = "p_value"
    Q_VALUE = "q_value"

class FDRMode(enum.Enum):
    PER_CHROMOSOME = "per_chromosome"
    WHOLE_GENOME = "whole_genome"

class PlotLibrary(enum.Enum):
    SEABORN = "seaborn"
    MATPLOTLIB = "matplotlib"
    PLOTLY = "plotly"

class BedSerializationMode(enum.Enum):
    MULTIPLE_CLUSTERS = "multiple_clusters"
    SINGLE_CLUSTER = "single_cluster"

# A single results table.
class ResultsTable:
    path : str
    df : pd.DataFrame

    def __init__(self, path, df):
        self.path = path
        self.df = df

    @staticmethod 
    def from_file(path) -> "ResultsTable":
        df = pd.read_csv(path, sep="\t")
        # add a column for the resolution
        df["resolution"] = df["name"].apply(lambda x: RESOLUTIONS[x.split("_")[0]])
        return ResultsTable(path, df)

    # A simple query system supporting the following functions
    # - less_than
    # - greater_than
    # - is_in
    # - is_not_in
    # - equals
    # - not_equals
    def where(self, key : str, *, less_than=None, greater_than=None, is_in=None, is_not_in=None, equals=None, not_equals=None, ) -> "ResultsTable":
        _df = self.df.copy()
        if less_than is not None:
            _df = _df[_df[key] < less_than]
        if greater_than is not None:
            _df = _df[_df[key] > greater_than]
        if is_in is not None:
            _df = _df[_df[key].isin(is_in)]
        if is_not_in is not None:
            _df = _df[_df[key].isin(is_not_in)]
        if equals is not None:
            _df = _df[_df[key] == equals]
        if not_equals is not None:
            _df = _df[_df[key] != not_equals]
        return ResultsTable(self.path, _df)

    @property
    def names(self):
        return self.df["name"].tolist()

    @property
    def size(self):
        return len(self.df)

    # overrides the jupyter notebook display to display the table
    def _repr_html_(self):
        return self.df.to_html()

    # Applies FDR
    def apply_fdr(self, mode : FDRMode) -> "ResultsTable":
        if mode == FDRMode.PER_CHROMOSOME:
            chromosomes = self.df["chromosome"].unique()
            _df = self.df.copy()
            for chromosome in chromosomes:
                _df.loc[_df["chromosome"] == chromosome, "q_value"] = sm.fdrcorrection(_df.loc[_df["chromosome"] == chromosome, "p_value"], alpha=0.05)[1]
            return ResultsTable(self.path, _df)
        
        if mode == FDRMode.WHOLE_GENOME:
            _df = self.df.copy()
            _df["q_value"] = sm.fdrcorrection(_df["p_value"], alpha=0.05)[1]
            return ResultsTable(self.path, _df)
    
    # plots the dataframe:
    def plot_chromosome_counts(self, key : str, plot_library : PlotLibrary, *, title=None, xlabel=None, ylabel=None, legend=True, **kwargs):
        if plot_library == PlotLibrary.PLOTLY:
            import plotly.graph_objs as go
            import plotly.express as px
            
            # plots an histogram of the entries for each chromosome
            fig = go.Figure()
            counts = self.df.groupby("chromosome")[key].count().to_dict()
            counts : dict = sort_by_chromosome(counts)
            fig.add_trace(go.Bar(x=list(counts.keys()), y=list(counts.values()), name="Counts"))
            fig.update_layout(title=title, xaxis_title=xlabel, yaxis_title=ylabel, legend_title_text="Chromosome")
            return fig

        else:
            raise NotImplementedError("Plotting library not implemented")

    def write_bed(self, path  :str, clusters : ClustersDescription, mode : BedSerializationMode = BedSerializationMode.MULTIPLE_CLUSTERS):
        
        if mode == BedSerializationMode.MULTIPLE_CLUSTERS:
            chromosomes = []
            starts = []
            ends = []
            names = []

            for row in self.df.itertuples():
                chromo = row.chromosome
                assert chromo == clusters.chromosome, "Chromosome mismatch, consider filtering before"

                cluster = clusters[row.name]
                bins = cluster.bins
                _ends = [s + int(cluster.resolution) for s in bins]

                chromosomes.extend([chromo] * len(bins))
                starts.extend(bins)
                ends.extend(_ends)
                names.extend([row.name] * len(bins))

            __df = pd.DataFrame({"chromosome": chromosomes, "start": starts, "end": ends, "name": names})
            __df.to_csv(path, sep="\t", index=False, header=False)

        else:
            with open(path, "W") as f:
                for row in self.df.itertuples():
                    chromo = row.chromosome
                    assert chromo == clusters.chromosome, "Chromosome mismatch, consider filtering before"
                    cluster = clusters[row.name]
                    bins = cluster.bins
                    _ends = [s + int(cluster.resolution) for s in bins]

                    
                    # Write the track name and the description, which contains the name of the cluster and the resolution
                    f.write(f"track name={row.name} description=\"{row.name}_{cluster.resolution}\"\n")

                    for s, e, n in zip(bins, _ends, [row.name] * len(bins)):
                        f.write(f"{chromo}\t{s}\t{e}\n")
                        
    
    def intersect(self, other : "ResultsTable") -> "ResultsTable":
        return ResultsTable(self.path, self.df.merge(other.df, on=["name", "chromosome"], ))

In [64]:
results_folder = "../results/"
CTCF_results_file = "HMEC.CTCF.tsv"
chromo = "chr16"

In [65]:
ctcf_results = ResultsTable.from_file(os.path.join(results_folder, CTCF_results_file)).where(ValueKeys.CHROMOSOME.value, equals=chromo).apply_fdr(FDRMode.PER_CHROMOSOME).where(ValueKeys.Q_VALUE.value, less_than=0.01)
cage_results = ResultsTable.from_file(os.path.join(results_folder, "HMEC.CAGE.tsv")).where(ValueKeys.CHROMOSOME.value, equals=chromo).apply_fdr(FDRMode.PER_CHROMOSOME).where(ValueKeys.Q_VALUE.value, less_than=0.01)

In [66]:
desc = ClustersDescription("../data/clusters/HMEC/chr16_spec_res.json", "chr16")

In [67]:
# Save the two files to a bed.
ctcf_results.write_bed("ctcf.bed", desc)
cage_results.write_bed("cage.bed", desc)

In [68]:
# Check the intersection
intersection = ctcf_results.intersect(cage_results)

In [70]:
# Save the intersection to a bed.
intersection.write_bed("intersection.bed", desc, mode=BedSerializationMode.SINGLE_CLUSTER)

In [71]:
def get_cg_content(sequence : str) -> float:
    return (sequence.count("C") + sequence.count("G")) / len(sequence)

In [72]:
data = "..."

In [73]:
get_cg_content(data)

500000